In [17]:
import torch 
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import sys
import copy

res = torch.load("processed/training.pt")
print(res[0].shape,res[1].shape)


class ToQuantumData(object):
    def __call__(self, tensor):
        # torch.set_printoptions(profile="full") 
        data = tensor
        input_vec = data.view(-1)
        vec_len = input_vec.size()[0]        
        input_matrix = torch.zeros(vec_len, vec_len)        
        input_matrix[0] = input_vec
                        
        input_matrix = input_matrix.transpose(0, 1)                
        u, s, v = np.linalg.svd(input_matrix)
        output_matrix = torch.tensor(np.dot(u, v))        
        output_data = output_matrix[:, 0].view(tensor.shape)        
        return output_data

trans_to_tensor = transforms.ToTensor()
trans_to_qc_data = ToQuantumData()
trans_resize = transforms.Resize((4,4))

data = torch.zeros(res[0].shape[0],4,4,dtype=torch.float32)


for i in range(res[0].shape[0]):
    # print(res[0][i].shape)
    if i%5000==0:
        print(i)
    npimg = res[0][i].numpy()   
    
    im = Image.fromarray(npimg,mode="L")
    qc_data = trans_to_qc_data(trans_to_tensor(trans_resize(im)))[0]    
    
    data[i] = qc_data    

torch.Size([60000, 28, 28]) torch.Size([60000])
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000


In [18]:
x = (data,res[1])
torch.save(x, 'processed/qc_training_4_4.pt')

In [19]:
res = torch.load("processed/qc_training_4_4.pt")
print(res[0][0])

tensor([[0.0149, 0.1843, 0.2641, 0.0947],
        [0.0199, 0.4185, 0.3936, 0.0548],
        [0.0149, 0.2242, 0.5132, 0.0448],
        [0.1594, 0.4285, 0.1694, 0.0050]])
